### Урок 3. Домашнее задание.

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

**Ответ:** для дисбалансной выборки лучше метрика precision_recall_curve. Например, в предложенном примере roc_auc будет практически одинаковый. Но если посчитать precision и recall, то всё встаёт на свои места - precision очень маленький во 2-ом случае (0.09, когда в 1-ом - 0.9) и это может быть существенно для принятия решения, хотя на roc_auc - это будет не заметно.

ссылка на соревнование - https://mlbootcamp.ru/ru/round/12/sandbox/

In [185]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, roc_auc_score

from scipy.sparse import hstack

import warnings

pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')

In [130]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [131]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделим наши данные на тренировочную и тестовую выборки

In [132]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], stratify=df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [133]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_ohe_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_ohe_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_ohe_transformers.append((cat_col, cat_ohe_transformer))

for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [134]:
feats_ohe = FeatureUnion(cat_ohe_transformers+base_transformers+continuos_transformers)
feats_forest = FeatureUnion(cat_ohe_transformers+base_transformers+continuos_transformers)

feature_processing = Pipeline([('feats_ohe', feats_ohe)])
feature_processing.fit_transform(X_train)

array([[ 1.        ,  0.        ,  0.        , ..., -0.36357212,
         0.01207798, -0.03585623],
       [ 1.        ,  0.        ,  1.        , ..., -0.77978795,
         0.0864655 , -0.03585623],
       [ 1.        ,  0.        ,  0.        , ..., -1.19600377,
         0.01207798, -0.08690066],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.39949022,
         0.01207798, -0.08690066],
       [ 1.        ,  0.        ,  1.        , ..., -1.05726516,
        -0.02511578, -0.03585623],
       [ 1.        ,  0.        ,  1.        , ...,  1.02381396,
         0.0864655 , -0.03585623]])

Добавим классификаторы

In [135]:
classifier_logreg = Pipeline([
    ('feats_ohe', feats_ohe),
    ('classifier', LogisticRegression(random_state = 42)),
])

classifier_knn = Pipeline([
    ('feats_ohe', feats_ohe),
    ('classifier', KNeighborsClassifier()),
])

classifier_xgb = Pipeline([
    ('feats_forest', feats_forest),
    ('classifier', xgb.XGBClassifier(random_state = 42)),
])

classifier_ranforest = Pipeline([
    ('feats_forest', feats_forest),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [187]:
params_ranforest = dict(classifier__n_estimators=[100, 200, 500],
                 classifier__max_depth=[3, 6, 8])
params_knn = dict(classifier__n_neighbors=[3, 5, 7, 10, 15])
params_logreg = dict(classifier__C=[0.5, 1, 10, 100])
params_xgb = dict(classifier__n_estimators=[100, 200, 500],
                 classifier__max_depth=[3, 6, 8])

Объединим поиск параметров и кросс-валидацию.

In [190]:
def find_parameters(pipe, params, X_train, y_train):
    
    rs = GridSearchCV(pipe, param_grid=params, scoring='roc_auc', cv=5)  # Тут же делаем кросс-валидацию
    rs.fit(X_train, y_train.values.ravel())
    
    print(f"Model: {pipe['classifier']},\n params: {rs.best_params_},\n best_score: {rs.best_score_}\n\n")

In [191]:
%%time
find_parameters(classifier_knn, params_knn, X_train, y_train)

Model: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform'), params: {'classifier__n_neighbors': 15}, best_score: 0.7242333904378239




In [192]:
%%time
find_parameters(classifier_ranforest, params_ranforest, X_train, y_train)

Model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False), params: {'classifier__max_depth': 8, 'classifier__n_estimators': 500}, best_score: 0.8002888605720699




In [194]:
%%time
find_parameters(classifier_logreg, params_logreg, X_train, y_train)

Model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), params: {'classifier__C': 100}, best_score: 0.7825536522091425


Wall time: 5.93 s


In [193]:
%%time
find_parameters(classifier_xgb, params_xgb, X_train, y_train)

Model: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None), params: {'classifier__max_depth': 3, 'classifier__n_estimators': 100}, best_score: 0.802224589239251


Wall time: 9min 28s


По 'ROC_AUC' победила модель XGBoost, теперь обучим все модели с подобранными параметрами.

In [196]:
classifier_logreg = Pipeline([
    ('feats_ohe', feats_ohe),
    ('classifier', LogisticRegression(C=100, random_state=42)),
])

classifier_knn = Pipeline([
    ('feats_ohe', feats_ohe),
    ('classifier', KNeighborsClassifier(n_neighbors=15)),
])

classifier_xgb = Pipeline([
    ('feats_forest', feats_forest),
    ('classifier', xgb.XGBClassifier(max_depth=3, n_estimators=100, random_state=42)),
])

classifier_ranforest = Pipeline([
    ('feats_forest', feats_forest),
    ('classifier', RandomForestClassifier(max_depth=8, n_estimators=500, random_state=42)),
])

In [197]:
%%time

#обучим пайплайн на всем тренировочном датасете
def fit_predict_models(pipe):
    pipe.fit(X_train, y_train)
    y_score = pipe.predict_proba(X_test)[:, 1]
    return y_score

y_score_logreg = fit_predict_models(classifier_logreg)
y_score_xgb = fit_predict_models(classifier_xgb)
y_score_knn = fit_predict_models(classifier_knn)
y_score_ranforest = fit_predict_models(classifier_ranforest)

Wall time: 25.6 s


Посчитаем precision/recall/f_score

In [201]:
def metrics_calculation(y_test, preds, b=1):

    roc_auc = roc_auc_score(y_test, preds)
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc

In [202]:
all_preds = [y_score_logreg, y_score_xgb, y_score_knn, y_score_ranforest]
metrics = []
for preds in all_preds:
    best_threshold, fscore, precision, recall, roc_auc = metrics_calculation(y_test, preds)
    metrics.append([fscore, precision, recall, roc_auc])

In [204]:
metrics_comparison_df = pd.DataFrame(np.array(metrics),
                   columns=['fscore', 'precision', 'recall', 'roc_auc'],
                   index=['y_score_logreg', 'y_score_xgb', 'y_score_knn', 'y_score_ranforest'])
metrics_comparison_df

,fscore,precision,recall,roc_auc
y_score_logreg,0.734072,0.665643,0.818182,0.784871
y_score_xgb,0.743349,0.697583,0.795540,0.802262
y_score_knn,0.703650,0.584398,0.884048,0.728406
y_score_ranforest,0.743088,0.687579,0.808348,0.800073


Лучше других по метрикам fscore и roc_auc себя показала модель XGBoost, а на втором месте случайный лес. При этом можно отметить, что если бы нас интересовал высокий Recall, то тут всех обошёл KNN, который по другим метрикам был в конце.
Это при том, что я даже не корректировал категорийные фичи под деревья - OHE для деревьев не оптимальный вариант. Скорее всего, результат был бы ещё лучше.